# Data Testing For Obj Detection

In [2]:
# my tool box for pytorch
from p3self.matchbox import *
from constant import *
from utils import *

In [3]:
%ls {IMG}|wc -l

    5000


In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import matplotlib.text as text
import numpy as np
import pandas as pd
import json
from glob import glob
from PIL import Image
import os

os.system("mkdir -p /data/bbsample")

0

In [6]:
imgdf=pd.DataFrame(jsdict["images"])
ann_df=pd.DataFrame(jsdict["annotations"])
urls = glob(IMG+"/*")
rdimg = np.random.choice(urls)
def get_id(url):
    return int(url.split("/")[-1].split(".")[0])

In [10]:
def df_data(imgdf,ann_df,shuffle=True):
    """
    imgdf:
    A dataframe about images, fields: "id","file_name","height","width"
    ann_df:
    A dataframe about annotation, fields: "image_id","category_id","bbox",
    The field "bbox" is a list of 4 values: x,y,height, width of the bounding box
    """
    data_df=pd.merge(ann_df[["bbox","category_id","image_id"]],
                     imgdf[["id","file_name","height","width"]],
                     left_on="image_id",right_on="id")
    
    data_df["cate_id_oh"] = data_df["category_id"].apply(lambda x:idx2id[x])
    if shuffle:
        data_df = data_df.sample(frac=1).reset_index(drop=True)
    print("total data rows",len(data_df))
    return data_df

data_df = df_data(imgdf,ann_df)
data_df.head()

total data rows 36781


,bbox,category_id,image_id,id,file_name,height,width,cate_id_oh
0,"[434.0, 38.54, 21.59, 61.31]",44,84241,84241,000000084241.jpg,396,640,39
1,"[253.62, 369.16, 12.12, 35.55]",1,93717,93717,000000093717.jpg,427,640,0
2,"[375.0, 126.69, 122.75, 125.0]",67,435081,435081,000000435081.jpg,500,500,60
3,"[383.88, 147.23, 116.12, 85.56]",63,468501,468501,000000468501.jpg,375,500,57
4,"[259.95, 189.77, 239.19, 143.23]",63,293804,293804,000000293804.jpg,333,500,57


In [11]:
bbox_array = np.array(data_df.bbox.tolist())
wh_array = data_df[["width","height"]].as_matrix()

def re_calibrate(bbox_array,wh_array):
    """return the resized bbox array"""
    bb_resized = (bbox_array/np.concatenate([wh_array,wh_array],axis=-1)) *SIZE
    
    true_bb = bb_resized/SCALE
    # switch xy as left top conner to center point
    true_bb[...,:2]=true_bb[...,:2]+true_bb[...,2:]/2
    # Labels' Anchor positions on the grid map
    grid_bbxy = np.floor(true_bb[...,:2])
    return bb_resized,true_bb,grid_bbxy

def find_best_anchors(true_bbwh):
    iou_score = []
    for b in range(BOX):
        wh_anc = np.tile(ANC_ARR[b],[true_bbwh.shape[0],1])
        true_area = true_bbwh.prod(axis=-1)
        anc_area = wh_anc.prod(axis=-1)
    
        inter_area = np.min([wh_anc,true_bbwh],axis=0).prod(axis=-1)
    
        union_area = true_area + anc_area - inter_area
        iou_score.append(inter_area/union_area)
    best_anchor_idx = np.array(iou_score).T.argmax(axis=-1)
    return best_anchor_idx

bb_resized,true_bb,grid_bbxy = re_calibrate(bbox_array,wh_array)
true_bbxy,true_bbwh = true_bb[...,:2],true_bb[...,2:]
best_anchor_idx = find_best_anchors(true_bbwh)

min_lbl = SCALE * 0.001

data_df["true_bb_x"],data_df["true_bb_y"],data_df["true_bb_w"],data_df["true_bb_h"]=true_bb[:,0],true_bb[:,1],true_bb[:,2],true_bb[:,3]
data_df["true_grid_x"],data_df["true_grid_y"]=grid_bbxy[:,0],grid_bbxy[:,1]

# data_df["true_bb_x"]=data_df["true_bb_x"]-data_df["true_grid_x"]
# data_df["true_bb_y"]=data_df["true_bb_y"]-data_df["true_grid_y"]

data_df["best_anchor"]=best_anchor_idx
data_df_ = data_df[data_df["true_bb_w"]>min_lbl]
data_df_ = data_df_[data_df_["true_bb_h"]>min_lbl]


In [34]:
a=np.random.rand(2,3,4,5)
a[0,1,2]=np.ones(5)

In [257]:
from torch.utils.data import dataset

from constant import *
import numpy as np

from PIL import Image
from torchvision import transforms

transform = transforms.Compose([transforms.Resize((HEIGHT,WIDTH)),
                                transforms.ToTensor(),
                                transforms.Normalize([.5,.5,.5],[.5,.5,.5])
                               ])
trans_origin = transforms.Compose([transforms.Resize((HEIGHT,WIDTH)),
                                transforms.ToTensor(),
                               ])
back2PIL = transforms.Compose([transforms.ToPILImage(mode="RGB")])

class Data_Multi(dataset.Dataset):
    def __init__(self,data_df,testing=False,*args,**kwargs):
        """
        Object detection data generatorcd 
        """
        super(Data_Multi,self).__init__()
        
        for k,v in kwargs.items():
            setattr(self,k,v)
        self.data_df = data_df
        self.img_ids = list(set(list(data_df["image_id"])))
        self.ids2fn = dict((k,v)for k,v in zip(self.data_df["image_id"],self.data_df["file_name"]))
        self.testing=testing
        self.anc = ANC_ARR
        
    def __len__(self):
        return len(self.img_ids)
        
    def __getitem__(self,idx):
        img=Image.open(self.id2url(self.img_ids[idx])).convert("RGB")

        sample = self.transform(img)
        
        original = self.trans_origin(img)
        
        img_df = self.data_df[self.data_df.image_id == self.img_ids[idx]].head(50)
        
        b_xywh = img_df[["true_bb_x","true_bb_y","true_bb_w","true_bb_h"]].as_matrix()
        posi = img_df[["true_grid_x","true_grid_y","best_anchor"]].as_matrix().astype(int)
        cls_id = img_df["cate_id_oh"].as_matrix()
        t_xywh = self.b2t_xy(b_xywh)
        t_xywh = self.b2t_wh(t_xywh,posi)
        
        N = t_xywh.shape[0]
        
        t_box = np.zeros((FEAT_W,FEAT_H,BOX,4))
        b_box = np.zeros((FEAT_W,FEAT_H,BOX,4))
        conf_ = np.zeros((FEAT_W,FEAT_H,BOX,1))
        cls_ = np.zeros((FEAT_W,FEAT_H,BOX,1))
        mask = np.zeros((FEAT_W,FEAT_H,BOX,1))
        cls_mask = np.zeros((FEAT_W,FEAT_H,BOX,1))
        
        for i_lbl in range(N):
            t_box[posi[i_lbl,0],posi[i_lbl,1],posi[i_lbl,2]] = t_xywh[i_lbl]
            b_box[posi[i_lbl,0],posi[i_lbl,1],posi[i_lbl,2]] = b_xywh[i_lbl]
            conf_[posi[i_lbl,0],posi[i_lbl,1],posi[i_lbl,2]] = 1
            cls_[posi[i_lbl,0],posi[i_lbl,1],posi[i_lbl,2]] = cls_id[i_lbl]
            mask[posi[i_lbl,0],posi[i_lbl,1],posi[i_lbl,2]] = 1
            cls_mask[posi[i_lbl,0],posi[i_lbl,1],posi[i_lbl,2]] = 1
            
        if self.testing:
            for i in sample,t_box,conf_,cls_,mask,cls_mask,b_box:
                print(i.shape)
        
        return sample,t_box,conf_,cls_,mask,cls_mask,b_box
    
    def get_id(self,url):
        return int(url.split("/")[-1].split(".")[0])
    
    def id2url(self,image_id):
        return IMG + self.ids2fn[image_id]
    
    def b2t_xy(self,x):
        x=x.copy()
        x[...,:2] = x[...,:2]-np.floor(x[...,:2])
        return x
    
    def b2t_wh(self,x,posi):
        x[...,2:4] = np.clip(x[...,2:4],1e-2,12.999)
        lb_s = x.shape[0]
        anc_tile=np.tile(self.anc[np.newaxis,:,:],[lb_s,1,1])
        x[...,2:4] = x[...,2:4]/anc_tile[np.eye(5)[posi[:,2]]==1]
        x[...,2:4] = np.log(x[...,2:4])
        return x    
    
    def true_adj_expand(self,true_adj):
        return  np.tile(true_adj[np.newaxis,np.newaxis,np.newaxis,:],[FEAT_W,FEAT_H,BOX,1])

In [258]:
ds = Data_Multi(data_df=data_df_,transform=transform,trans_origin=trans_origin)

In [259]:
dl = DataLoader(ds,batch_size=2,shuffle=True)
gen = iter(dl)

In [260]:
sample,t_box,conf_,cls_,mask,cls_mask,b_box=next(gen)

In [261]:
b_box[...,0:1][mask==1]


  2.5410
  4.9585
  7.0332
  8.1505
 10.3306
  2.0008
  2.4166
  6.1232
  6.5000
  7.3190
  7.8752
  9.2298
 10.8544
 10.9960
 11.5073
[torch.DoubleTensor of size 15]

In [246]:
Variable(mask).double()==0

Variable containing:
(0 ,0 ,0 ,.,.) = 
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1

(0 ,0 ,1 ,.,.) = 
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1

(0 ,0 ,2 ,.,.) = 
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
      ...

(0 ,0 ,10,.,.) = 
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1

(0 ,0 ,11,.,.) = 
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1

(0 ,0 ,12,.,.) = 
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
        ⋮ 

(0 ,1 ,0 ,.,.) = 
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1

(0 ,1 ,1 ,.,.) = 
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1

(0 ,1 ,2 ,.,.) = 
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
  1  1  1  1  1
      ...

(0 ,1 ,10,.,.) = 
  1  1  1  1  1
  1  1  1  1  1
  1  1 

In [95]:
dict((k,v)for k,v in zip(data_df_["image_id"],data_df_["file_name"]))

{84241: '000000084241.jpg',
 93717: '000000093717.jpg',
 435081: '000000435081.jpg',
 468501: '000000468501.jpg',
 293804: '000000293804.jpg',
 69138: '000000069138.jpg',
 302030: '000000302030.jpg',
 494869: '000000494869.jpg',
 136915: '000000136915.jpg',
 120853: '000000120853.jpg',
 580418: '000000580418.jpg',
 122166: '000000122166.jpg',
 105923: '000000105923.jpg',
 79969: '000000079969.jpg',
 456015: '000000456015.jpg',
 188465: '000000188465.jpg',
 293794: '000000293794.jpg',
 342128: '000000342128.jpg',
 56350: '000000056350.jpg',
 210520: '000000210520.jpg',
 179265: '000000179265.jpg',
 78823: '000000078823.jpg',
 212800: '000000212800.jpg',
 77460: '000000077460.jpg',
 163057: '000000163057.jpg',
 350023: '000000350023.jpg',
 493905: '000000493905.jpg',
 65288: '000000065288.jpg',
 348481: '000000348481.jpg',
 256192: '000000256192.jpg',
 484351: '000000484351.jpg',
 166521: '000000166521.jpg',
 455716: '000000455716.jpg',
 303818: '000000303818.jpg',
 61333: '000000061333.

In [140]:
posi = ds.data_df[["true_grid_x","true_grid_y","best_anchor"]].head(20).as_matrix()

In [148]:
np.eye(5)[posi[:,2].astype(int)]

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])